# LRCStats - Long Read Correction Statistics #
---

## Background ##
---
DNA sequencing is the process of constructing a string of characters taken from the set {A,C,G,T} (representing the nucleotide bases adenine, cytosine, guanine and thymine) to represent the order of nucleotides that compose a DNA molecule. Genomes, however, cannot be sequenced in full - only segments, or "reads" as we will call it, can be sequenced at a time. Currently, there are two commonly used sequencing technologies - Illumina cyclic reversible termination (CRT) systems, which output short, accurate reads and Pacific Biosciences single-molecule real-time (SMRT) technologies, which output long, inaccurate reads. (Goodwin et al, 2016) We will refer to reads outputted by Illumina CRT systems as "Illumina reads" and reads outputted by Pacific BioSciences SMRT systems as "PacBio reads".

Illumina short reads, as the name suggests, are short in length, with read lengths ranging from 100 to 150 base pairs (bp) depending on the system used. In contrast, the length of the _Eshercheria chia_ genome, a bacterial species with a relatively simple genome, is approximately 4 000 000 bp. (cite this) However, the error rate of Illumina reads is < 1% at most, with errors being mostly substitutions. (Goodwin et al, 2016) 

PacBio reads, on the other hand, are much longer than Illumina short reads, with mean lengths of 20kb for the Pacific BioSciences RS II system. However, the error rate profile is quite concerning - 13% single pass error rate with the majority of errors being insertions and deletions. (Goodwin et al, 2016)

Despite the low error rate of Illumina reads, the short lengths constrain the usage of Illumina short reads in genome assembly. This is because genomes commonly contain repeat sequences - regions of the genome that are similar, if not the same, to prior regions of the genome. If the input reads do not fully span the entire length of these repeats, this can cause the assembled genome to be fragmented, which is not ideal for downstream analysis. (cite this) The long lengths of PacBio reads are thus preferable for genome assembly as they can consolidate the majority of repeats in a genome and allow for longer, contiguous assembled genomes. However, the high error rate of PacBio reads can cause problems in downstream analysis as well.

### Long Read Correction Algorithms ###
Long read correction algorithms are a class of algorithms that attempt to reduce the error rate of long reads.
_Hybrid correction algorithms_ are a subset of these algorithms. The idea behind hybrid correction algorithms is to correct erroneous long reads with accurate short reads, thus creating long and highly accurate reads, which is ideal for genome assembly.

To correct long reads, recent hybrid correction algorithms follow two methods:

1. Map the short reads onto the long reads using a mapper, such as Burrows-Wheeler Aligner
2. Construct a de Bruijn graph using the short reads and align the long reads onto the graph structure

**Recently published hybrid correction algorithms**

| Name      | Date |
| :-------: | :--: |
| Proovread | 2014 |
| LoRDEC    | 2014 |
| Jabba     | 2016 |
| CoLoRMap  | 2016 |

## Methods ## 
---

![LRCStats pipeline](lrcstats_pipeline.png)

### Overview ###
Long Read Correction Statistics (LRCStats) is an open source pipeline that benchmarks long read correction algorithms using simulated long read data. The overall goal of LRCStats is to assess the accuracy of correction algorithms with simulated data where errors have been precisely identified. The current LRCStats pipeline is split into four distinct steps:

#### 1. Simulate Long and Short Reads ####
Currently, LRCStats uses SimLoRD and ART to simulate PacBio long and Illumina short reads, respectively. LRCStats is designed to be easily modified to use other long read simulators if the user so desires.

The long read simulator SimLoRD outputs simulated long reads in a FASTQ file format along with a SAM alignment file that specifies the true alignment of the simulated long reads with the reference genome. We then convert this SAM file into a [MAF](https://genome.ucsc.edu/FAQ/FAQformat.html#format5) file. Whereas SAM files represent the alignment of a long read and the reference sequence in the CIGAR format (cite this), MAF files directly represent the alignment of the two sequences. This allows us to align the corrected read onto the original alignment using a dynamic programming algorithm and perform statistics on this new three-way alignment.

#### 2. Correct Long Reads ####
LRCStats currently supports four hybrid correction algorithms: Proovread, LoRDEC, Jabba and CoLoRMap. The default parameters for each algorithm were chosen to best reflect common usage. LRCStats is designed to be easily modified to accomodate new long read correction algorithms, and not only hybrid correction algorithms.

#### 3. Construct Three-Way Alignments ####
Using the MAF file outputted in step 1, LRCStats aligns the corrected long read (cLR) onto the original reference (ref) and uncorrected long read alignment (uLR) to create a three-way alignment. This is done using a specially-designed dynamic programming algorithm that takes into account the structure of the corrected long reads outputted by the four hybrid correction algorithms in use.

The hybrid correction algorithms can fail to correct certain segments of the simulated long reads. Each algorithm handles this differently. CoLoRMap and LoRDEC indicate these uncorrected segments in the corrected long reads by outputting uncorrected bases in lowercase, whereas corrected bases are outputted in uppercase. Proovread "trims" these uncorrected segments from the corrected long reads to output only the corrected segments of the original uncorrected read. Jabba does not currently keep note of which bases in the corrected long read have been corrected, and may split the read into separate subreads. The alignment algorithm takes into account both cases when creating the three-way alignment.

#### 4. Collect Statistics on the Three-Way Alignments ####
The three-way alignment outputted in the previous step allows us to collect statistics in two different ways:

1. On the entire length of the read
2. On only those segments of the reads that have been corrected

This provides us a general method to compare the accuracy of corrected long reads outputted by the various algorithms. The statistics currently outputted by LRCStats are:

1. Error rate over all bases
2. Throughput, i.e. total number of outputted bases
3. Number of correct and incorrect bases
4. Number of deletions, insertions and substitutions among the incorrect bases

The above statistics are collected for both corrected and uncorrected reads. As well, these statistics are categorized into two other separate categories, which are: 

1. Statistics collected on the entire length of the three-way alignment
2. Statistics collected on only those segments of the corrected read which have been corrected and the corresponding segment of the uncorrected read

We call the former category the "untrimmed" statistics and the latter the "trimmed" statistics.

### Implementation Details ###

#### Programming Languages Used ####
Except for the aliger program, LRCStats is written in Python 2.7 for ease of maintenance and usage for the user. The aligner program is written in C++ 11 to take advantage of the speed of this compiled programming language. This is especially important for sequence alignments which can require a significant amount of time to compute despite having polynomial time complexity.

#### Code Structure ####
The code for this project is organized into the following main directories:

* `config`: contains the configuration files for different experiments
* `notebooks`: contains notebooks detailing the experiments featured in the LRCStats paper
* `pipeline`: contains Python modules for the construction of the PBS scripts for each experiment
* `scripts`: contains the scripts for each experiment
* `src`: contains Python scripts and C++ source code for processing, aligning and analyzing the corrected long reads

#### Alignment Algorithm ####
The sequence alignment algorithm used for the creation of the three-way alignments of the reference sequence, uncorrected and corrected long reads is based on the common dynamic programming algorithm for calculating the minimum edit distance between two sequences. We modified this algorithm in two ways:

1. Our algorithm constructs optimal local alignments by either:
  * anchoring uncorrected segments of untrimmed corrected long reads to the corresponding identical segments in the uncorrected long reads 
  * allowing for unpenalized deletions between trimmed segments of trimmed corrected long reads
2. Our algorithm aligns a sequence to an alignment of two other sequences (i.e. we align the corrected long read with the _alignment_ of the uncorrected long read and the reference sequence).

Recall that the correction algorithms Jabba and Proovread output trimmed long reads and CoLoRMap and LoRDEC output untrimmed long reads.

Given a corrected long read C[1..n] and an alignment A[1..m] of the reference sequence and uncorrected long read, the time complexity of this algorithm is then O(nm).

## Results ##
---

### Data ###

### Experiment Design ###

### Experiment Results ###

## Conclusion ##
---

## Appendix ##
---

### LRCStats Manual ###
See it [here](manual.md).

### Scripts Used in Featured Experiment ###